<a href="https://colab.research.google.com/github/sab112/myfirstrepo/blob/master/myGridSearchCV_RamdomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Finding best model and hyper parameter tunning using GridSearchCV and RandomizedCSearchCV

from sklearn import svm, datasets
import pandas as pd

iris = datasets.load_iris()
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])

In [2]:
#Approach 1: Use train_test_split and manually tune parameters by trial and error
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9111111111111111

In [4]:
#Approach 2: Use K Fold Cross validation
#Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation.
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [5]:

cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [7]:
#Approach 3: Use GridSearchCV
#GridSearchCV loop above cross_val_score but in a single line of code

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'), {'C': [1,10,20],'kernel': ['rbf','linear']}, cv=5, return_train_score=False)

clf.fit(iris.data, iris.target)

clf.cv_results_

{'mean_fit_time': array([0.00126014, 0.0004425 , 0.00055289, 0.0004746 , 0.00059462,
        0.00045328]),
 'mean_score_time': array([0.00062199, 0.00027518, 0.00029807, 0.00027852, 0.0003109 ,
        0.00028992]),
 'mean_test_score': array([0.98      , 0.98      , 0.98      , 0.97333333, 0.96666667,
        0.96666667]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'rank_test_score': array([1, 1, 1, 4, 5, 6], dtype=int32),
 'split0_test_score': array([0.96666667, 0.96

In [8]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001260,0.000457,0.000622,0.000261,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000443,0.000029,0.000275,0.000011,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000553,0.000030,0.000298,0.000011,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000475,0.000026,0.000279,0.000019,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000595,0.000029,0.000311,0.000021,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000453,0.000010,0.000290,0.000027,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [9]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [ ]:
dir(clf)

In [11]:
clf.best_params_  #gives params that makes best fit, so that we can use this further 

{'C': 1, 'kernel': 'rbf'}

In [14]:
#Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. 
#This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {'C': [1,10,20],'kernel': ['rbf','linear']},cv=5, return_train_score=False, n_iter=2)  
#gives only 2(n_iter), even though we have 6 in combinations, run again to get next random params provided

rs.fit(iris.data, iris.target)

pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,20,linear,0.966667
